In [ ]:
# Качаем данные о всех регионах России
import pyrosm
from pyrosm.data import sources

directory = "D:/Topology Service/Urban-Topology-Analysis-Service/experiments/data/"
cityName = "Московская область"

for region in sources.europe.russia.available:
    pyrosm.get_data(region, update=False, directory=directory)

In [ ]:
# Административные границы РФ (их типы)
import pyrosm
import geopandas
import numpy as np

for i in range(3, 11):
    print(f'LEVEL: {i}')
    try:
        filePath = f'D:/Topology Service/Urban-Topology-Analysis-Service/experiments/boarders/admin_level_{i}.geojson' 
        dataframe = geopandas.read_file(filePath, driver='GeoJSON')

        dataframe = dataframe.astype({'official_status': 'str'})
        status = np.unique(dataframe['official_status'])
        print(f'STATUS: {status}')
    except Exception:
        print('None')


In [ ]:
import pyrosm
import geopandas
import numpy as np

level = 6
filePath = f'D:/Topology Service/Urban-Topology-Analysis-Service/experiments/boarders/admin_level_{level}.geojson' 
dataframe = geopandas.read_file(filePath, driver='GeoJSON')

dataframe = dataframe.astype({'official_status': 'str'})
dataframe = dataframe[dataframe['name'] == 'городской округ Красноярск']
# print(np.unique(dataframe['name']))
dataframe.plot()

# osm = pyrosm.OSM(filePath, bounding_box=)


# dataframe = osm.get_boundaries()
# dataframe.plot(column=dataframe['name'], cmap='viridis') 


# levels = np.unique(dataframe['admin_level'])
# print(f'LEVELS: {levels}')

# result = dataframe[dataframe['admin_level'] == 4]
# dataframe.plot()
 
# result_str = ''
# for name in result['name']:
#     result_str += " "+name+"; "

# print(result_str)

# nodes, edges = osm.get_network(network_type="driving", nodes=True)
# drive_net = osm.get_network(network_type="driving")
# drive_net.plot(figsize=(12,12))

In [ ]:
import pyrosm

directory = 'D:/Topology Service/Urban-Topology-Analysis-Service/experiments/data/siberian-fed-district-latest.osm.pbf'

bbox_geom = dataframe['geometry'].values[0]
osm = pyrosm.OSM('D:/Topology Service/Urban-Topology-Analysis-Service/experiments/data/siberian-fed-district-latest.osm.pbf', bounding_box=bbox_geom)

camden = osm.get_buildings()
print('buildings')
ax = camden.plot(column="building", figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=3, bbox_to_anchor=(1, 1)))

In [ ]:
#Extract certain area

selected_boundary = osm.get_boundaries(name="Василеостровский район")
selected_boundary.plot(facecolor="none", edgecolor="blue")
bbox_geom = selected_boundary['geometry'].values[0]

osm = pyrosm.OSM(filepath=filePath, bounding_box=bbox_geom)


In [ ]:
#Extract graph and buildings

drive_net = osm.get_network(network_type="driving")
ax = drive_net.plot(figsize=(12,12), lw=0.7, alpha=0.6, color="orange")
camden = osm.get_buildings()
camden.plot(ax=ax)

In [ ]:
#Список городов РФ

import pandas as pd

path = './cities.csv'

df = pd.read_csv(path)
df.head(100)
# df.to_csv('./cities.csv')

In [ ]:
import osmnx as ox
def exists(city):
    query = {'city': city}
    try:
        gdf = ox.geocode_to_gdf(query)
        return True
    except ValueError:
        return False

i = 0
for city in df['Город']:
    print(i)
    i+=1
    if not exists(city):
        print(city)

In [ ]:
import osmnx as ox
import pandas
import geopandas
import numpy as np


dataframe = geopandas.read_file('./regions.json', driver='GeoJSON')
cities = pandas.read_csv('./cities.csv')

# def uniq(lst):
#     last = object()
#     for item in lst:
#         if item == last:
#             continue
#         yield item
#         last = item

# def sort_and_deduplicate(l):
#     return list(uniq(sorted(l, reverse=True)))

def children(ids_list, buffer):
    plot(ids_list=ids_list)
    buffer.extend(ids_list)
    children = dataframe[dataframe['parents'].str.contains('|'.join(str(x) for x in ids_list), na=False)]
    return children['osm_id'].to_list()

def plot(ids_list):
    if len(ids_list) != 0:
        children = dataframe[dataframe['osm_id'].isin(ids_list)]
        test = children.iloc[0]['geometry']
        # print(test)
        print(test.boundary)
        # print(test.coords)
        # children.plot(cmap='viridis')



# # for city in cities[cities['Население'] > 1000000]['Город']:
# #     df = dataframe[dataframe['local_name']==city]
# #     ids_list = df['osm_id'].to_list()

# #     while len(ids_list) != 0:
# #         plot(ids_list=ids_list, city=city)
# #         ids_list = children(ids_list=ids_list, buffer=buffer)

buffer = []
# no_found = set()

for city in cities[cities['Население'] < 100000]['Город']:
    df = dataframe[dataframe['local_name']==city]
    ids_list = df['osm_id'].to_list()
   
    while len(ids_list) != 0:
        ids_list = children(ids_list=ids_list, buffer=buffer)

# buffer = sort_and_deduplicate(buffer)

# print(buffer)
# print(no_found)

# df = dataframe[dataframe['osm_id'].isin(buffer)]
# df.to_file('./regions_new.json', driver="GeoJSON")  
# ct = cities[~cities['Город'].isin(no_found)]
# ct.to_csv('./cities_new.csv')

In [ ]:
# exctract all highways

import osmium as o

filePath = 'D:/Topology Service/Urban-Topology-Analysis-Service/experiments/data/test.osm' 

nodes_ids = set()
ways_ids = set()

class PreConvert(o.SimpleHandler):
    def way(self, w):
        if 'highway' in w.tags:
            ways_ids.add(w.id)
            for node in w.nodes:
                nodes_ids.add(node.ref)

class PostConvert(o.SimpleHandler):
    def __init__(self, writer):
        super(PostConvert, self).__init__()
        self.writer = writer

    def node(self, n):
        if n.id in nodes_ids:
            self.writer.add_node(n)

    def way(self, w):
        if w.id in ways_ids:
            self.writer.add_way(w)



handler = PreConvert()
handler.apply_file(filePath)

In [ ]:
writer = o.SimpleWriter('./highways.osm')
handler = PostConvert(writer)
handler.apply_file(filePath)
writer.close()

In [ ]:
import osmnx as ox
import numpy as np

filePath = 'D:/Topology Service/Urban-Topology-Analysis-Service/experiments/highways.osm' 
G = ox.graph_from_xml(filePath)

In [ ]:
import osmnx as ox
import pandas 
import geopandas

# def check(city, df):
#     query = {'city': city}
#     try:
#         gdf = ox.geocode_to_gdf(query)
#         df = pd.concat([df, gdf[['geometry', 'bbox_north', 'bbox_south', 'bbox_east',  'bbox_west']]])
#         return df
#     except ValueError:
#         return df

filePath = './cities.csv' 
dataframe = pandas.read_csv(filePath)
dataframe
# dataframe = dataframe[dataframe['Население']>1000000]
# cities = []
# df = pd.DataFrame(columns=['city', 'geometry', 'bbox_north', 'bbox_south', 'bbox_east',  'bbox_west'])
# for city in dataframe['Город']:
#     cities.append(city)
#     df = check(city, df)

# df['city'] = cities

# df.head()


In [ ]:
import pandas


dataframe = geopandas.read_file('./regions.json', driver='GeoJSON')

dataframe = dataframe[dataframe['local_name']=='Москва']
test = dataframe.iloc[0]['geometry']
pol = str(test.boundary)
print(dataframe)


with open("Output.txt", "w") as text_file:
    text_file.write(pol)


In [ ]:
import geopandas as gpd
from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry.linestring import LineString

def to_list(polygon : LineString):
    list = []
    for x, y in polygon.coords:
        list.append([x, y])
    x, y = polygon.coords[0]
    list.append([x, y])
    return list


def to_json_array(polygon):
    coordinates_list = []
    if type(polygon) == LineString:
       coordinates_list.append(to_list(polygon))
    elif type(polygon) == MultiLineString:
        for line in polygon:
            coordinates_list.append(to_list(line))
    else:
        raise ValueError("polygon must be type of LineString or MultiLineString")

    return coordinates_list

def region_to_scheme(regions, ids_list, depth : int):
    polygons = regions[regions['osm_id'].isin(ids_list)]
    ids_list = regions[regions['parents'].str.contains('|'.join(str(x) for x in ids_list), na=False)]['osm_id'].to_list()
    for index, row in polygons.iterrows():
        print(to_json_array(row['geometry'].boundary))


    print('!'*100)
    return ids_list

def regions_to_scheme(city, regions):
    ids_list = regions[regions['local_name']==city]['osm_id'].to_list()
    depth = 0
    while len(ids_list) != 0:
        ids_list = region_to_scheme(regions=regions, ids_list=ids_list, depth=depth)
        depth += 1
        

# regions_df = gpd.read_file('./regions.json', driver='GeoJSON') 
regions_to_scheme('Севастополь', regions_df)

